In [7]:
#pip install icrawler

In [8]:
from icrawler.builtin import BingImageCrawler, GoogleImageCrawler, BaiduImageCrawler

import os

# List of cattle breeds
breeds = [
    "Holstein cow", "Jersey cow", "Angus cow", "Brahman cow", "Hereford cow",
    "Simmental cow", "Limousin cow", "Guernsey cow", "Charolais cow", "Ayrshire cow"
]

# Directory to save images
base_dir = "cattle_dataset"
os.makedirs(base_dir, exist_ok=True)

# Download ~500 images per breed
for breed in breeds:
    folder_name = breed.replace(" ", "_")
    save_path = os.path.join(base_dir, folder_name)
    os.makedirs(save_path, exist_ok=True)
    total_images_saved = 0  # Initialize total images saved for the breed

    # Bing images
    crawler = BingImageCrawler(storage={'root_dir': save_path})
    crawler.crawl(keyword=breed, max_num=1000)
    print(f"Downloaded images for {breed}")

    # Google images
    google_crawler = GoogleImageCrawler(storage={'root_dir': save_path})
    google_crawler.crawl(keyword=breed, max_num=1000)
    print(f"Downloaded ~1000 images for {breed} from Google")

    # Baidu images
    baidu_crawler = BaiduImageCrawler(storage={'root_dir': save_path})
    baidu_crawler.crawl(keyword=breed, max_num=1000)   
    print(f"Downloaded ~1000 images for {breed} from Baidu")



2025-06-12 05:52:48,492 - INFO - icrawler.crawler - start crawling...


2025-06-12 05:52:48,493 - INFO - icrawler.crawler - starting 1 feeder threads...
2025-06-12 05:52:48,535 - INFO - icrawler.crawler - starting 1 parser threads...
2025-06-12 05:52:48,555 - INFO - icrawler.crawler - starting 1 downloader threads...
2025-06-12 05:52:49,271 - INFO - parser - parsing result page https://www.bing.com/images/async?q=Holstein cow&first=0
2025-06-12 05:52:49,370 - INFO - downloader - skip downloading file 000001.jpg
2025-06-12 05:52:49,373 - INFO - downloader - skip downloading file 000002.jpg
2025-06-12 05:52:49,374 - INFO - downloader - skip downloading file 000003.jpg
2025-06-12 05:52:49,375 - INFO - downloader - skip downloading file 000004.jpg
2025-06-12 05:52:49,376 - INFO - downloader - skip downloading file 000005.jpg
2025-06-12 05:52:49,376 - INFO - downloader - skip downloading file 000006.jpg
2025-06-12 05:52:49,377 - INFO - downloader - skip downloading file 000007.jpg
2025-06-12 05:52:49,378 - INFO - downloader - skip downloading file 000008.jpg
20

Downloaded images for Holstein cow


2025-06-12 05:53:17,142 - INFO - downloader - downloader-001 is waiting for new download tasks
2025-06-12 05:53:19,013 - INFO - parser - parsing result page https://www.google.com/search?q=Holstein+cow&ijn=0&start=0&tbs=&tbm=isch
Exception in thread parser-001:
Traceback (most recent call last):
  File "C:\Users\Mansal\AppData\Local\Programs\Python\Python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\Mansal\AppData\Local\Programs\Python\Python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "d:\pylearn\GroupProject_1\ISB46703\lib\site-packages\icrawler\parser.py", line 104, in worker_exec
    for task in self.parse(response, **kwargs):
TypeError: 'NoneType' object is not iterable

2025-06-12 05:53:22,147 - INFO - downloader - no more download task for thread downloader-001
2025-06-12 05:53:22,147 - INFO - downloader - thread downloader-001 exit
2025-06-12 05:53:22,319 - INFO - icrawler.crawler - Crawling ta

Downloaded ~1000 images for Holstein cow from Google


...

Downloaded images for Ayrshire cow


2025-06-12 05:59:45,541 - INFO - downloader - downloader-001 is waiting for new download tasks
2025-06-12 05:59:47,360 - INFO - parser - parsing result page https://www.google.com/search?q=Ayrshire+cow&ijn=0&start=0&tbs=&tbm=isch
Exception in thread parser-001:
Traceback (most recent call last):
  File "C:\Users\Mansal\AppData\Local\Programs\Python\Python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\Mansal\AppData\Local\Programs\Python\Python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "d:\pylearn\GroupProject_1\ISB46703\lib\site-packages\icrawler\parser.py", line 104, in worker_exec
    for task in self.parse(response, **kwargs):
TypeError: 'NoneType' object is not iterable

2025-06-12 05:59:50,551 - INFO - downloader - no more download task for thread downloader-001
2025-06-12 05:59:50,552 - INFO - downloader - thread downloader-001 exit
2025-06-12 05:59:50,692 - INFO - icrawler.crawler - Crawling ta

Downloaded ~1000 images for Ayrshire cow from Google


2025-06-12 05:59:51,264 - INFO - parser - parsing result page http://image.baidu.com/search/acjson?tn=resultjson_com&ipn=rj&word=Ayrshire cow&pn=0&rn=30
Exception in thread parser-001:
Traceback (most recent call last):
  File "C:\Users\Mansal\AppData\Local\Programs\Python\Python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\Mansal\AppData\Local\Programs\Python\Python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "d:\pylearn\GroupProject_1\ISB46703\lib\site-packages\icrawler\parser.py", line 104, in worker_exec
    for task in self.parse(response, **kwargs):
  File "d:\pylearn\GroupProject_1\ISB46703\lib\site-packages\icrawler\builtin\baidu.py", line 120, in parse
    for item in content['data']:
KeyError: 'data'

2025-06-12 05:59:55,789 - INFO - downloader - no more download task for thread downloader-001
2025-06-12 05:59:55,791 - INFO - downloader - thread downloader-001 exit
2025-06-12 05:59:55,885 - I

Downloaded ~1000 images for Ayrshire cow from Baidu


In [9]:
from PIL import Image
import os

def resize_images(input_folder):
    for folder in os.listdir(input_folder):
        folder_path = os.path.join(input_folder, folder)
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            try:
                img = Image.open(img_path)
                img = img.resize((224, 224))
                img.save(img_path)
            except:
                os.remove(img_path)  # remove corrupted images

resize_images("cattle_dataset")


In [10]:
import os
import shutil
from sklearn.model_selection import train_test_split

src_folder = "cattle_dataset"
dst_folder = "dataset"
splits = ['train', 'val', 'test']
split_ratios = [0.7, 0.15, 0.15]  # 70% train, 15% val, 15% test

for breed in os.listdir(src_folder):
    breed_path = os.path.join(src_folder, breed)
    images = os.listdir(breed_path)
    train, temp = train_test_split(images, test_size=0.3, random_state=42)
    val, test = train_test_split(temp, test_size=0.5, random_state=42)

    for split, files in zip(splits, [train, val, test]):
        split_path = os.path.join(dst_folder, split, breed)
        os.makedirs(split_path, exist_ok=True)
        for file in files:
            shutil.copy(os.path.join(breed_path, file), os.path.join(split_path, file))


In [11]:
import json

# Prepare a dictionary to store the dataset structure
dataset_info = {}

for split in splits:
    split_dir = os.path.join(dst_folder, split)
    dataset_info[split] = {}
    for breed in os.listdir(split_dir):
        breed_dir = os.path.join(split_dir, breed)
        images = os.listdir(breed_dir)
        dataset_info[split][breed] = images

# Save the dataset structure to a JSON file
with open(os.path.join(dst_folder, "dataset_info.json"), "w") as f:
    json.dump(dataset_info, f, indent=2)


print("Dataset structure saved to dataset_info.json")



Dataset structure saved to dataset_info.json
